# LangChain 实战：房产销售聊天机器人

你是顶级的电脑配件销售，现在培训职场新人，请给出100条实用的销售回答。
每条话术如下格式产出：
1.
[新人问题]
[回答]

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [82]:
with open("real_estate_sales_data.txt", encoding='utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [83]:
from langchain.text_splitter import CharacterTextSplitter

In [84]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [85]:
docs = text_splitter.create_documents([real_estate_sales])

In [86]:
docs[0]

Document(page_content='[新人问题] 客户问我电脑配件的兼容性问题时我该怎么回答？\n[回答] 首先确认客户的电脑型号和现有配置，然后为他们解释我们的产品兼容性，并提供专业建议。')

In [87]:
len(docs)

32

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [88]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [89]:
query = "配件接上后电脑运行不起来"

In [90]:
answer_list = db.similarity_search(query)

In [91]:
for ans in answer_list:
    print(ans.page_content + "\n")

[新人问题] 客户对电脑组装服务有疑问。
[回答] 详细介绍我们的组装服务，包括价格、所需时间和服务保证。

[新人问题] 客户问我电脑配件的兼容性问题时我该怎么回答？
[回答] 首先确认客户的电脑型号和现有配置，然后为他们解释我们的产品兼容性，并提供专业建议。

[新人问题] 如果客户对配件的性能表示怀疑怎么办？
[回答] 提供性能测试报告或用户评价，必要时可以提供试用服务。

[新人问题] 如果客户急需配件但我们缺货怎么办？
[回答] 向客户道歉并解释缺货的原因，提供最快的补货时间或帮助他们找到其他供应商。



In [102]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [93]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [94]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002184BBB2140>, search_kwargs={'k': 3})

In [95]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[新人问题] 客户对电脑组装服务有疑问。
[回答] 详细介绍我们的组装服务，包括价格、所需时间和服务保证。

[新人问题] 客户问我电脑配件的兼容性问题时我该怎么回答？
[回答] 首先确认客户的电脑型号和现有配置，然后为他们解释我们的产品兼容性，并提供专业建议。

[新人问题] 如果客户对配件的性能表示怀疑怎么办？
[回答] 提供性能测试报告或用户评价，必要时可以提供试用服务。



In [96]:
docs = topK_retriever.get_relevant_documents("你们有没有1000万的豪宅啊？")

In [97]:
for doc in docs:
    print(doc.page_content + "\n")

[新人问题] 客户问我们是否有教育或批量购买折扣。
[回答] 介绍任何针对教育机构或大量购买的特别折扣计划。

[新人问题] 客户对价格敏感，怎么办？
[回答] 强调性价比，提供不同价格区间的选项，并解释长远来看投资高质量配件的好处。

[新人问题] 如果客户需要的配件我们没有怎么回答？
[回答] 诚实告知客户，并提供预订服务或推荐替代产品，保持良好的客户关系。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [98]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [99]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

c:\ProgramData\Anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


### 提取向量数据库中的`销售回答`

In [100]:
docs = retriever.get_relevant_documents(query)

c:\ProgramData\Anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [101]:
docs[0].page_content

IndexError: list index out of range

In [ ]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 我担心楼下太吵。\n', '这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']

In [ ]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [ ]:
ans

'这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'

#### 尝试各种问题

In [ ]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [ ]:
query = "我想离医院近点"

print(sales(query))

[]


c:\ProgramData\Anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [ ]:
print(sales(query, 0.75))

['有的，距离我们小区不远就有几家大型综合医院。', '是的，附近有多家大型医院，医疗资源非常丰富。']


In [ ]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

c:\ProgramData\Anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



c:\ProgramData\Anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['我们有不同户型和付款方案，一定有适合您预算的。', '我们的房子位于黄金地段，升值潜力很大，转卖起来不会有问题。', '定金金额取决于您选择的房型和付款方式，我们可以详细为您解释。', '这个区域正在快速发展，未来的升值潜力非常大。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [ ]:
qa_chain({"query": "你们小区有200万的房子吗？"})

c:\ProgramData\Anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
c:\ProgramData\Anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们小区有200万的房子吗？', 'result': '对不起，我不知道。我没有关于房地产的信息。'}

In [ ]:
qa_chain({"query": "小区吵不吵"})

{'query': '小区吵不吵',
 'result': '这个小区特别注重居住体验，有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。所以应该不会太吵。'}

In [ ]:
print(sales("小区吵不吵"))

['这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']


## 加载 FAISS 向量数据库已有结果

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

c:\ProgramData\Anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
qa_chain({"query": "我想买别墅，你们有么"})

c:\ProgramData\Anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买别墅，你们有么', 'result': '对不起，我不能帮助你购买别墅。我是一个人工智能，我主要用于提供信息和回答问题。'}

In [ ]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [ ]:
qa_chain({"query": "我想买别墅，你们有么"})

c:\ProgramData\Anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么',
 'result': '很抱歉，作为一个AI助手，我无法直接卖房子或提供房地产服务。但是，我可以提供一些建议或回答关于购买别墅的问题。如果您有关于购买别墅的问题，请随时提问。'}

In [ ]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [ ]:
result = qa_chain({"query": "我想买别墅，你们有么"})

c:\ProgramData\Anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [ ]:
result

{'query': '我想买别墅，你们有么',
 'result': '抱歉，作为一个AI助手，我无法直接帮您购买别墅。但我可以为您提供一些建议和信息，帮助您了解如何购买别墅。如果您有关于购买别墅的问题，请随时提问。',
 'source_documents': []}